# Create an interface between two materials with minimal strain

Use Zur and McGill superlattices matching [algorithm](https://doi.org/10.1063/1.3330840) to create interfaces between two materials using the Pymatgen [implementation](https://pymatgen.org/pymatgen.analysis.interfaces.html#pymatgen.analysis.interfaces.zsl).

## Usage

1.  Specify parameters (e.g. `MAX_AREA`) below or use the default values
2.  Click "Run" > "Run All" to run all cells
3.  Review the strain plot and modify its parameters as needed

## Methodology

The following happens in the script below:

1. Create slabs for each input material. The materials data is passed in from and back to the web application according to the description [here](TBA).
   We assume that two input materials are either in the bulk form (e.g. Ni crystal) or layered (e.g. graphene).

2. Perform strain matching on the slabs to extract the supercell dimensions. The algorithm has a set of parameters, such as the maximum area considered, that can be configured by editing the cells below.

3. When the strain matching is finished, the interface with the lowest strain (and the smallest number of atoms) is selected. We create the corresponding supercells and place them at a specified distance from each other (note no shift is performed currently).


## Parameters

### The algorithm parameters

The following parameters are used for ZSL


In [ ]:
ZSL_PARAMETERS = {
    "MAX_AREA": 400,  # The area to consider in Angstrom^2
    "MAX_AREA_TOL": 0.09,  # The area within this tolerance is considered equal
    "MAX_LENGTH_TOL": 0.03,  # supercell lattice vectors lengths within this tolerance are considered equal
    "MAX_ANGLE_TOL": 0.01,  # supercell lattice angles within this tolerance are considered equal
    "STRAIN_TOL": 10e-6,  # strains within this tolerance are considered equal
}

### Select Materials

We set which material is considered "substrate" (bottom) and which "layer" (top) using the indexes from the list of materials passed. Additionally, we also set miller indices and thickness of the corresponding slabs.


In [ ]:
SUBSTRATE_PARAMETERS = {
    "MATERIAL_INDEX": 0,  # the index in the list of input materials
    "MILLER_INDICES": (1, 1, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 3,  # in layers
}

LAYER_PARAMETERS = {
    "MATERIAL_INDEX": 1,  # the index in the list of input materials
    "MILLER_INDICES": (0, 0, 1),  # the miller indices of the interfacial plane
    "THICKNESS": 1,  # in layers
}

## Interface Parameters

The resulting interface will be set accordingly


In [ ]:
INTERFACE_PARAMETERS = {"DISTANCE_Z": 3.0}  # in Angstrom

In [ ]:
# Install the required packages
from ..utils import install_packages

await install_packages()
from development.definitions import get_materials, set_materials, to_pymatgen, from_pymatgen, plot_strain_vs_atoms

# Get the list of input materials and load them into `data` variable
get_materials()

## Create interfaces

All possible combinations within the maximum area considered are produced.


In [ ]:
from development.pymatgen_coherent_interface_builder import CoherentInterfaceBuilder, ZSLGenerator
from operator import itemgetter

pymatgen_materials = [to_pymatgen(item) for item in data]
for material in pymatgen_materials:
    print(material)


def create_interfaces(settings):
    # Interface Builder class
    zsl = ZSLGenerator(
        max_area_ratio_tol=settings["ZSL_PARAMETERS"]["MAX_AREA_TOL"],
        max_area=settings["ZSL_PARAMETERS"]["MAX_AREA"],
        max_length_tol=settings["ZSL_PARAMETERS"]["MAX_LENGTH_TOL"],
        max_angle_tol=settings["ZSL_PARAMETERS"]["MAX_ANGLE_TOL"],
    )

    cib = CoherentInterfaceBuilder(
        substrate_structure=pymatgen_materials[settings["SUBSTRATE_INDEX"]],
        film_structure=pymatgen_materials[settings["LAYER_INDEX"]],
        substrate_miller=settings["SUBSTRATE_PARAMETERS"]["MILLER_INDICES"],
        film_miller=settings["LAYER_PARAMETERS"]["MILLER_INDICES"],
        zslgen=zsl,
        strain_tol=settings["ZSL_PARAMETERS"]["STRAIN_TOL"],
    )

    # Run the Interface Building process
    cib._find_terminations()
    matches = cib.zsl_matches
    terminations = cib.terminations

    # Create interfaces
    interfaces = []
    for termination in terminations:
        interfaces = list(
            cib.get_interfaces(
                termination,
                gap=settings["INTERFACE_PARAMETERS"]["DISTANCE_Z"],
                film_thickness=settings["LAYER_PARAMETERS"]["THICKNESS"],
                substrate_thickness=settings["SUBSTRATE_PARAMETERS"]["THICKNESS"],
                in_layers=True,
            )
        )

    print(f"Found {len(matches)} interfaces")
    print(f"Found {len(terminations)} terminations:", terminations)
    return interfaces


interfaces = create_interfaces(
    settings={**SUBSTRATE_PARAMETERS, **LAYER_PARAMETERS, **ZSL_PARAMETERS, **INTERFACE_PARAMETERS}
)

## Sort interfaces by strain

We use mean absolute strain and sort the output accordingly.


In [ ]:
strain_modes = {
    "VON_MISES": "von_mises_strain",
    "STRAIN": "strain",
    "MEAN": "mean_abs_strain",
}
strain_mode = strain_modes["MEAN"]
interfaces_list = list(interfaces)

# Sort interfaces by ascending strain and then by ascending number of atoms
sorted_interfaces = sorted(
    interfaces_list,
    key=lambda x: (itemgetter(strain_mode)(x), x["interface"].num_sites),
)

print("Interface with lowest strain (index 0):")
print("    strain:", sorted_interfaces[0][strain_mode] * 100, "%")
print("    number of atoms:", sorted_interfaces[0]["interface"].num_sites)

## Plot the number of atoms vs strain

Adjust the plot parameters as needed.


In [ ]:
PLOT_SETTINGS = {
    "X_MIN": 0.01,  # percentage
    "X_MAX": 100,  # percentage
    "Y_MIN": 1,  # number of atoms
    "Y_MAX": 1000,  # number of atoms
    "X_SCALE": "log",
    "Y_SCALE": "log",
}

plot_strain_vs_atoms(strain_mode, sorted_interfaces, settings=PLOT_SETTINGS)

## Select interface with minimal strain

As below


In [ ]:
selected_interface = from_pymatgen(interfaces[0]["interface"])

## Pass to the outside runtime

As below


In [ ]:
output_materials = [selected_interface]
set_materials(output_materials)